Import Dependencies

In [ ]:
import json
import time
import datetime
import boto3
import io
import pandas as pd
import trade29
import matplotlib as plt
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import mplfinance as mpf
from trade29.sc import constants
from trade29.sc.bridge import SCBridge
from trade29.sc.constants import OrderStatusCode

Read the JSON file for settings

In [ ]:
# read JSON file for settings
with open('C:\\Users\\bgroe\\OneDrive\\Desktop\\Algo_config.json') as config_file:
    config = json.load(config_file)

file_path = config["file_path"]
specific_day = config["specific_day"]
trade_key = config['trade_key']
sleep_interval = config['sleep_interval']
full_trade_quantity = config['full_trade_quantity']
First_oco_quantity = config['first_OCO_quantity']
Second_oco_quantity = config['Second_OCO_quantity']
Third_oco_quantity = config['Third_OCO_quantity']
Deltabot_interval = config['Deltabot_interval']
csv_directory_delta = config['file_path_delta']
risk_per_trade = config['Risk_per_trade']
tick_value = config['Tick_value']
Trade_time_open = config['Trade_time_open']
Trade_time_close = config['Trade_time_close']
Flatten_positions_time = config['Flatten_positions_time']
Maximum_loss_per_day = config['Maximum_loss_per_day']
Profit_target = config['Profit_target']
Deltabot_downloadtime = config['Deltabot_downloadtime']

In [ ]:
webhook_url = 'discord_webhook_minute'
webhook_url_hourly = 'discord_webhook_hour'
columns_to_drop = [' OHLC Avg', ' HLC Avg', ' HL Avg', ' Bid Volume',
                   ' Ask Volume', ' Sum DownLvl Values (BAR)',
                  ' Counter DownLvl Values (BAR)', ' First DownLvl Value (BAR)',
                  ' Counter UpLvl Values (BAR)', ' Sum UpLvl Values (BAR)',
                   ' First UpLvl Value (BAR)']
sg_columns_to_drop = [f' SG{i}' for i in range(1, 61)]
columns_to_drop = columns_to_drop + sg_columns_to_drop
column_names_delta = ['Date', 'Open']
global_df = pd.DataFrame()
final_dict_down = {}
final_dict_up = {}
global_df['Count Before Down'] = 0
global_df['Count Before Up'] = 0
liquidity_gap_above = []
liquidity_gap_below = []
liquidity_side_accepted = True

filepath_zones = 'file_path_zones'

global target1_parent, target1_target, target1_stop
global target2_parent, target2_target, target2_stop
global target3_parent, target3_target, target3_stop
global Risk_reward, First_oco_target, Second_oco_target, Stoploss_price

target1_parent, target1_target, target1_stop = None, None, None
target2_parent, target2_target, target2_stop = None, None, None
target3_parent, target3_target, target3_stop = None, None, None

First_oco_target, Second_oco_target, Stoploss_price = None, None, None

global last_message_time
last_message_time = datetime.now() - timedelta(minutes=1)

Function to download the csv file from amazon web

In [ ]:

def download_optionbot():
    try:
        session = boto3.Session(
            aws_access_key_id='aws_id',
            aws_secret_access_key='aws_key',
            region_name='aws_region'
        )
        s3 = session.resource('s3')
        BUCKET = "d-sig-bucket"
        s3.Bucket(BUCKET).download_file("optiondelta.txt", csv_directory_delta)

    except Exception as e:
        print("Error downloading the optionbot CSV file:", e)

Function to read the CSV file and update the global DataFrame

In [ ]:
# Function to read the CSV file and update the global DataFrame
def read_csv():
    global global_df, current_time, option_delta, zone_df
    try:
        df = pd.read_csv(file_path)
        df = df.drop(columns=columns_to_drop)
        df = df[df['Date'] == specific_day]
        global_df = df
        now = datetime.now()
        option_delta = pd.read_csv(csv_directory_delta, names=column_names_delta, skiprows=1)
        option_delta['Date'] = pd.to_datetime(option_delta['Date'], format='%H:%M:%S')
        option_delta['Date'] = option_delta['Date'].dt.floor('min')
        option_delta = option_delta.drop_duplicates(subset='Date', keep='first')
        zone_df = pd.read_csv(filepath_zones)
        zone_df = zone_df.drop(index=0).reset_index(drop=True)

        current_time = now.strftime("%Y-%m-%d %H:%M:%S")
        print(
            "========================================================================================================")
        print("Current date and time of last update:", current_time)
        print(
            "--------------------------------------------------------------------------------------------------------")
    except Exception as e:
        print("Error reading the CSV file:", e)

Function to calculate the Rate of Change of the extracted Option delta

In [ ]:
def Option_delta_calculation():
    global option_delta
    try:
        roc_period = 15
        roc_values = option_delta['Open'].diff(roc_period) / option_delta['Open'].shift(roc_period).abs()
        option_delta['ROC_15min'] = roc_values
        option_delta['Delta'] = 0

        # Calculate delta
        for i in range(2, len(option_delta)):
            y1 = option_delta.iloc[i - round(roc_period/2)]['ROC_15min']
            y2 = option_delta.iloc[i]['ROC_15min']

            delta = y2 - y1

            option_delta.iloc[i, option_delta.columns.get_loc('Delta')] = delta

        # Assign initial position as 'Hold'
        option_delta['Position'] = 'Hold'

        # Determine position based on ROC and slope angle
        for i in range(1, len(option_delta)):

            if option_delta.iloc[i]['ROC_15min'] < 0 and option_delta.iloc[i][
                'Delta'] < 0:
                option_delta.iloc[i, option_delta.columns.get_loc('Position')] = 'Short'

            elif option_delta.iloc[i]['ROC_15min'] > 0 and option_delta.iloc[i][
                'Delta'] > 0:
                option_delta.iloc[i, option_delta.columns.get_loc('Position')] = 'Long'

    except Exception as e:
        print("Error calculating deltabot:", e)


In [ ]:
def calculate_zones():
    global global_df
    try:
        if global_df.empty:
            print("Global DataFrame is empty")
            return

        # Calculate zones above price
        filtered_df_up = global_df[global_df[' First UpLvl Price (BAR)'] != 0]
        if filtered_df_up.empty:
            print("No rows with 'First UpLvl Price (BAR)' not equal to 0")
        else:
            liquidity_gap_above.clear()
            liquidity_gap_below.clear()
            result_dict_up = filtered_df_up.set_index(['Date', ' Time'])[' First UpLvl Price (BAR)'].to_dict()

            global_df['DateTime'] = pd.to_datetime(global_df['Date'] + ' ' + global_df[' Time'])

            for (date, time), price in result_dict_up.items():
                date_time = pd.to_datetime(date + ' ' + time)
                high_after_time = global_df[global_df['DateTime'] > date_time][' High'].max()

                reached_level = high_after_time >= price

                if time not in final_dict_up:
                    final_dict_up[time] = {}
                final_dict_up[time][price] = {
                    'Date': date,
                    'Reached Level': reached_level,
                    'High After Time': high_after_time
                }

            unreached_levels_up = [price for time, prices in final_dict_up.items() for price, data in prices.items() if
                                   not data['Reached Level']]
            unreached_levels_up.sort()

            if unreached_levels_up:
                for i in range(len(unreached_levels_up) - 1):
                    if unreached_levels_up[i + 1] - unreached_levels_up[i] > 0.25:
                        gap = unreached_levels_up[i]
                        liquidity_gap_above.append(gap)
                        # print(f"Liquidity gaps above price: {unreached_levels_up[i]} and {unreached_levels_up[i + 1]}")
            else:
                print("No liquiditygaps above price")

        # Calculate zones below price
        filtered_df_down = global_df[global_df[' First DownLvl Price (BAR)'] != 0]
        if filtered_df_down.empty:
            print("No rows with 'First DownLvl Price (BAR)' not equal to 0")
        else:
            result_dict_down = filtered_df_down.set_index(['Date', ' Time'])[' First DownLvl Price (BAR)'].to_dict()

            global_df['DateTime'] = pd.to_datetime(global_df['Date'] + ' ' + global_df[' Time'])

            for (date, time), price in result_dict_down.items():
                date_time = pd.to_datetime(date + ' ' + time)
                low_after_time = global_df[global_df['DateTime'] > date_time][' Low'].min()

                reached_level = low_after_time <= price

                if time not in final_dict_down:
                    final_dict_down[time] = {}
                final_dict_down[time][price] = {
                    'Date': date,
                    'Reached Level': reached_level,
                    'Low After Time': low_after_time
                }

            unreached_levels_down = [price for time, prices in final_dict_down.items() for price, data in prices.items()
                                     if not data['Reached Level']]
            unreached_levels_down.sort(reverse=True)

            if unreached_levels_down:
                for i in range(len(unreached_levels_down) - 1):
                    if unreached_levels_down[i] - unreached_levels_down[i + 1] > 0.25:
                        gap = unreached_levels_down[i]
                        liquidity_gap_below.append(gap)
                        # print(f"Liquidity gaps below price: {unreached_levels_down[i+1]} and {unreached_levels_down[i]}")
            else:
                print("No Liquiditygaps found below price")
        # print("--------------------------------------------------------------------------------------------------------")
    except Exception as e:
        print("Error calculating zones:", e)

Function to calculate which side of the liquidity is accepted

In [ ]:
def calculate_liquidity_side():
    global liquidity_side_accepted, global_df, final_dict_down, final_dict_up
    try:
        # Convert time strings to datetime.time objects for comparison
        global_df['Time_dt'] = pd.to_datetime(global_df[' Time'], format=' %H:%M:%S.%f').dt.time
        reached_levels_down = [(time, price) for time, prices in final_dict_down.items() for price, data in
                               prices.items() if data['Reached Level']]
        reached_levels_up = [(time, price) for time, prices in final_dict_up.items() for price, data in prices.items()
                             if data['Reached Level']]
        # Pre-compute time comparisons
        reached_levels_down_times = [datetime.strptime(time, ' %H:%M:%S.%f').time() for time, _ in reached_levels_down]
        reached_levels_up_times = [datetime.strptime(time, ' %H:%M:%S.%f').time() for time, _ in reached_levels_up]

        # Add a new column for Liquidity Side
        global_df['Liquidity Side'] = True

        # Ensure the DataFrame has a RangeIndex for proper iteration
        global_df = global_df.reset_index(drop=True)
        global_df['Extension'] = 'Normal Action'
        global_df['Normal Action Count'] = 0

        # Flag to track liquidity side status
        liquidity_side = True

        # Iterate through the DataFrame rows
        for idx in range(1, len(global_df)):

            if global_df[' RSI'].iloc[idx] >= 60:
                global_df.at[idx, 'Extension'] = 'Extended'

            elif global_df[' RSI'].iloc[idx] <= 40:
                global_df.at[idx, 'Extension'] = 'Extended'

            if global_df['Extension'].iloc[idx] == 'Normal Action':
                if global_df['Extension'].iloc[idx - 1] == 'Extended':
                    global_df.at[idx, 'Normal Action Count'] = 1
                else:
                    global_df.at[idx, 'Normal Action Count'] = global_df['Normal Action Count'].iloc[idx - 1] + 1



            Liquidity_trend = global_df.at[idx, ' Liquidity Trend']

            # Check if cumulative increase in 'Count Before Down' is greater than 2
            if Liquidity_trend == -1:
                global_df.at[idx, 'Liquidity Side'] = False
                liquidity_side = False

            # Check if cumulative increase in 'Count Before Up' is greater than 2 when liquidity side is False
            elif Liquidity_trend == 1:
                global_df.at[idx, 'Liquidity Side'] = True
                liquidity_side = True

            else:
                # Propagate the current liquidity side value
                global_df.at[idx, 'Liquidity Side'] = liquidity_side

        if global_df['Liquidity Side'].iloc[-1] == True and option_delta['Position'].iloc[-2] == 'Long':
            global_df.at[global_df.index[-1], 'Bias'] = ':green_circle: Long'
        elif global_df['Liquidity Side'].iloc[-1] == False and option_delta['Position'].iloc[-2] == 'Short':
            global_df.at[global_df.index[-1], 'Bias'] = ':red_circle: Short'
        else:
            global_df.at[global_df.index[-1], 'Bias'] = ':white_circle: Hold'

        if global_df['Liquidity Side'].iloc[-13] == True and option_delta['Position'].iloc[-3] == 'Long':
            global_df.at[global_df.index[-13], 'Bias'] = ':green_circle: Long'
        elif global_df['Liquidity Side'].iloc[-13] == False and option_delta['Position'].iloc[-3] == 'Short':
            global_df.at[global_df.index[-13], 'Bias'] = ':red_circle: Short'
        else:
            global_df.at[global_df.index[-13], 'Bias'] = ':white_circle: Hold'


    except Exception as e:
        print("Error calculating liquidity side:", e)


Actual trading brain, calculate risk and amount of contracts

In [ ]:
def calculate_risk_management():
    global target1_parent, target1_target, target1_stop, Stoploss_price, First_oco_target, Second_oco_target, Third_oco_target
    global target2_parent, target2_target, target2_stop
    global target3_parent, target3_target, target3_stop
    global Risk_reward
    try:

        Management_condition = 0
        Trade_condition = 0

        Position_quantity = global_df[' Position Quantity'].iloc[-1]
        latest_open = global_df[' Open'].iloc[-1]
        unreached_levels_down = [price for time, prices in final_dict_down.items() for price, data in prices.items() if
                                 not data['Reached Level']]
        unreached_levels_down.sort(reverse=True)
        unreached_levels_down = pd.Series(unreached_levels_down).drop_duplicates().tolist()

        unreached_levels_up = [price for time, prices in final_dict_up.items() for price, data in prices.items() if
                               not data['Reached Level']]
        unreached_levels_up.sort()
        unreached_levels_up = pd.Series(unreached_levels_up).drop_duplicates().tolist()

        closest_below_price = unreached_levels_down[0]
        second_below_price = unreached_levels_down[1]
        third_below_price = unreached_levels_down[2]

        closest_above_price = unreached_levels_up[0]
        second_above_price = unreached_levels_up[1]
        third_above_price = unreached_levels_up[2]

        difference_up_zone = closest_above_price - latest_open
        difference_risk_zone_up = third_above_price - latest_open

        difference_down_zone = latest_open - closest_below_price
        difference_risk_zone_down = latest_open - third_below_price

        closest_liq_gap_up = liquidity_gap_above[0]
        second_liq_gap_up = liquidity_gap_above[1]
        difference_gap_up = closest_liq_gap_up - latest_open
        difference_second_gap_up = second_liq_gap_up - latest_open

        closest_liq_gap_down = liquidity_gap_below[0]
        second_liq_gap_down = liquidity_gap_below[1]
        difference_gap_down = latest_open - closest_liq_gap_down
        difference_second_gap_down = latest_open - second_liq_gap_down

        Parent_order_price = latest_open
        if global_df['Liquidity Trend V2'].iloc[-1] == 'Long':
            First_oco_target = closest_above_price
            Second_oco_target = closest_liq_gap_up
            Parent_order_price = closest_below_price
            Stoploss_price = Parent_order_price - 6
            Risk_reward = difference_up_zone / difference_risk_zone_down
            if difference_second_gap_up < 15:
                Third_oco_target = second_liq_gap_up
            else:
                Third_oco_target = closest_liq_gap_up

        elif global_df['Liquidity Trend V2'].iloc[-1] == 'Short':
            First_oco_target = closest_below_price
            Second_oco_target = closest_liq_gap_down
            Parent_order_price = closest_above_price
            Stoploss_price = Parent_order_price + 6
            Risk_reward = difference_down_zone / difference_risk_zone_up
            if difference_second_gap_down < 15:
                Third_oco_target = second_liq_gap_up
            else:
                Third_oco_target = closest_liq_gap_up

        if (global_df['Liquidity Trend V2'].iloc[-1] == 'Long' and option_delta['Position'].iloc[-2] == 'Long' and global_df[' Avg'].iloc[-1] >= 70
            and global_df['Extension'].iloc[-1] == 'Normal Action' and global_df['Normal Action Count'].iloc[-1] >= 24):
                Trade_condition = 1

        elif (global_df['Liquidity Trend V2'].iloc[-1] == 'Short' and option_delta['Position'].iloc[ -2] == 'Short' and global_df[' Avg'].iloc[-1] >= 70
            and global_df['Extension'].iloc[-1] == 'Normal Action' and global_df['Normal Action Count'].iloc[-1] >= 24):
                Trade_condition = 2

        oco_list = [
            trade29.sc.bridge.AttachedOrder(quantity=First_oco_quantity, target_price=First_oco_target,
                                            stop_price=Stoploss_price),
            trade29.sc.bridge.AttachedOrder(quantity=Second_oco_quantity, target_price=Second_oco_target,
                                            stop_price=Stoploss_price),
            trade29.sc.bridge.AttachedOrder(quantity=Third_oco_quantity, target_price=Third_oco_target,
                                            stop_price=Stoploss_price)]

        if Position_quantity == 0:
            if target1_parent is not None:
                if bridge.get_order_status(trade_key, target1_parent).order_status_code == OrderStatusCode.OPEN:
                    if round_to_quarters(bridge.get_order_status(trade_key, target1_parent).price1) != Parent_order_price:
                        bridge.modify_order(trade_key, order_id=target1_parent, quantity=First_oco_quantity, price1=Parent_order_price)
                        bridge.modify_order(trade_key, order_id=target2_parent, quantity=Second_oco_quantity, price1=Parent_order_price)
                        bridge.modify_order(trade_key, order_id=target3_parent, quantity=Third_oco_quantity, price1=Parent_order_price)

            if Trade_condition == 0:
                if target1_parent is not None:
                    if bridge.get_order_status(trade_key, target1_parent).order_status_code == OrderStatusCode.OPEN:
                        bridge.cancel_order(trade_key, order_id=target1_parent)
                        bridge.cancel_order(trade_key, order_id=target2_parent)
                        bridge.cancel_order(trade_key, order_id=target3_parent)

                    elif bridge.get_order_status(trade_key, target2_parent).order_status_code == OrderStatusCode.OPEN:
                        bridge.cancel_order(trade_key, order_id=target2_parent)
                    elif bridge.get_order_status(trade_key, target3_parent).order_status_code == OrderStatusCode.OPEN:
                        bridge.cancel_order(trade_key, order_id=target3_parent)

            if Trade_condition == 1:
                if target1_parent is not None:
                    if bridge.get_order_status(trade_key, target1_parent).order_status_code != OrderStatusCode.OPEN:
                        response = bridge.submit_order(trade_key, is_buy=True, quantity=full_trade_quantity,
                                                       order_type=constants.OrderType.LIMIT, price1=Parent_order_price,
                                                       ocos=oco_list)
                        print("Placing buy order")
                        print(f"Target 1: {difference_up_zone} points")
                        print(f"Target 2: {difference_gap_up} points")
                        print(f"Target 3: {difference_second_gap_up} points")
                        print(f"Stop: {difference_risk_zone_down} points")

                        target1_parent = response.order_groups[0].parent_id
                        target1_target = response.order_groups[0].target_id
                        target1_stop = response.order_groups[0].stop_id

                        target2_parent = response.order_groups[1].parent_id
                        target2_target = response.order_groups[1].target_id
                        target2_stop = response.order_groups[1].stop_id

                        target3_parent = response.order_groups[2].parent_id
                        target3_target = response.order_groups[2].target_id
                        target3_stop = response.order_groups[2].stop_id

                else:
                    response = bridge.submit_order(trade_key, is_buy=True, quantity=full_trade_quantity,
                                                   order_type=constants.OrderType.LIMIT, price1=Parent_order_price,
                                                   ocos=oco_list)
                    print("Placing buy order")
                    print(f"Target 1: {difference_up_zone} points")
                    print(f"Target 2: {difference_gap_up} points")
                    print(f"Target 3: {difference_second_gap_up} points")
                    print(f"Stop: {difference_risk_zone_down} points")

                    target1_parent = response.order_groups[0].parent_id
                    target1_target = response.order_groups[0].target_id
                    target1_stop = response.order_groups[0].stop_id

                    target2_parent = response.order_groups[1].parent_id
                    target2_target = response.order_groups[1].target_id
                    target2_stop = response.order_groups[1].stop_id

                    target3_parent = response.order_groups[2].parent_id
                    target3_target = response.order_groups[2].target_id
                    target3_stop = response.order_groups[2].stop_id

            if Trade_condition == 2:
                if target1_parent is not None:
                    if bridge.get_order_status(trade_key, target1_parent).order_status_code != OrderStatusCode.OPEN:
                        response = bridge.submit_order(trade_key, is_buy=False, quantity=full_trade_quantity,
                                                       order_type=constants.OrderType.LIMIT, price1=Parent_order_price,
                                                       ocos=oco_list)
                        print("Placing sell order")
                        print(f"Target 1: {difference_down_zone} points")
                        print(f"Target 2: {difference_gap_down} points")
                        print(f"Target 3: {difference_second_gap_down} points")
                        print(f"Stop: {difference_risk_zone_up} points")

                        target1_parent = response.order_groups[0].parent_id
                        target1_target = response.order_groups[0].target_id
                        target1_stop = response.order_groups[0].stop_id

                        target2_parent = response.order_groups[1].parent_id
                        target2_target = response.order_groups[1].target_id
                        target2_stop = response.order_groups[1].stop_id

                        target3_parent = response.order_groups[2].parent_id
                        target3_target = response.order_groups[2].target_id
                        target3_stop = response.order_groups[2].stop_id
                else:
                    response = bridge.submit_order(trade_key, is_buy=False, quantity=full_trade_quantity,
                                                   order_type=constants.OrderType.LIMIT, price1=Parent_order_price,
                                                   ocos=oco_list)
                    print("Placing sell order")
                    print(f"Target 1: {difference_down_zone} points")
                    print(f"Target 2: {difference_gap_down} points")
                    print(f"Target 3: {difference_second_gap_down} points")
                    print(f"Stop: {difference_risk_zone_up} points")

                    target1_parent = response.order_groups[0].parent_id
                    target1_target = response.order_groups[0].target_id
                    target1_stop = response.order_groups[0].stop_id

                    target2_parent = response.order_groups[1].parent_id
                    target2_target = response.order_groups[1].target_id
                    target2_stop = response.order_groups[1].stop_id

                    target3_parent = response.order_groups[2].parent_id
                    target3_target = response.order_groups[2].target_id
                    target3_stop = response.order_groups[2].stop_id

        if Position_quantity != 0:
            if Position_quantity > 0:
                if Trade_condition == 2 or global_df['Liquidity Trend V2'].iloc[-1] == 'Short':
                    Management_condition = 1

                elif (round_to_quarters(bridge.get_order_status(trade_key, target1_target).price1) > First_oco_target and
                    bridge.get_order_status(trade_key, order_id =target1_target).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 2

                elif (round_to_quarters(bridge.get_order_status(trade_key, target2_target).price1) > Second_oco_target and
                    bridge.get_order_status(trade_key, order_id =target2_target).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 3

                elif (round_to_quarters(bridge.get_order_status(trade_key, target3_target).price1) > Third_oco_target and
                    bridge.get_order_status(trade_key, order_id =target3_target).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 4

                elif (round_to_quarters(bridge.get_order_status(trade_key, target1_stop).price1) < Stoploss_price and
                    bridge.get_order_status(trade_key, order_id =target1_stop).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 5
                    stopprice = Stoploss_price

                elif (round_to_quarters(bridge.get_order_status(trade_key, target2_stop).price1) < Stoploss_price and
                    bridge.get_order_status(trade_key, order_id =target2_stop).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 5
                    stopprice = Stoploss_price

                elif (round_to_quarters(bridge.get_order_status(trade_key, target3_stop).price1) < Stoploss_price and
                    bridge.get_order_status(trade_key, order_id =target2_stop).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 5
                    stopprice = Stoploss_price

                elif bridge.get_order_status(trade_key, target1_target).order_status_code == OrderStatusCode.FILLED:
                    Management_condition = 6
                else:
                    Management_condition = 0

            if Position_quantity < 0:
                if Trade_condition == 1 or global_df[' Liquidity Trend V2'].iloc[-1] == 'Long':
                    Management_condition = 1

                elif (round_to_quarters(bridge.get_order_status(trade_key, target1_target).price1) > First_oco_target and
                    bridge.get_order_status(trade_key, order_id =target1_target).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 2

                elif (round_to_quarters(bridge.get_order_status(trade_key, target2_target).price1) > Second_oco_target and
                    bridge.get_order_status(trade_key, order_id =target2_target).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 3

                elif (round_to_quarters(bridge.get_order_status(trade_key, target3_target).price1) > Third_oco_target and
                    bridge.get_order_status(trade_key, order_id =target3_target).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 4

                elif (round_to_quarters(bridge.get_order_status(trade_key, target1_stop).price1) > Stoploss_price and
                    bridge.get_order_status(trade_key, order_id =target1_stop).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 5
                    stopprice = Stoploss_price

                elif (round_to_quarters(bridge.get_order_status(trade_key, target2_stop).price1) > Stoploss_price and
                    bridge.get_order_status(trade_key, order_id =target2_stop).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 5
                    stopprice = Stoploss_price

                elif (round_to_quarters(bridge.get_order_status(trade_key, target3_stop).price1) > Stoploss_price and
                    bridge.get_order_status(trade_key, order_id =target2_stop).order_status_code == OrderStatusCode.OPEN):
                    Management_condition = 5
                    stopprice = Stoploss_price

                elif bridge.get_order_status(trade_key, order_id=target1_target).order_status_code == OrderStatusCode.FILLED:
                    Management_condition = 6

                else:
                    Management_condition = 0

        if Position_quantity != 0:
            if Management_condition == 0:
                pass
            elif Position_quantity > 0:
                BE_price = round_to_quarters(bridge.get_order_status(trade_key, target1_parent).average_fill_price)

                if Management_condition == 1:
                    bridge.flatten_and_cancel(trade_key)
                    print("Flatten & canceled position ")

                elif Management_condition == 2:
                    # change target attached order
                    if bridge.get_order_status(trade_key, order_id =target1_target).order_status_code == OrderStatusCode.OPEN:
                        bridge.modify_order(trade_key, order_id = target1_target, quantity= First_oco_quantity , price1= First_oco_target)
                        print("Change target attached order 1 ")

                elif Management_condition == 3:
                    # change target 2 attached order

                    if bridge.get_order_status(trade_key, order_id = target2_target).order_status_code == OrderStatusCode.OPEN:
                        bridge.modify_order(trade_key, order_id = target2_target, quantity = Second_oco_quantity, price1= Second_oco_target)
                        print("Change target 2 attached order 2")

                elif Management_condition == 4:
                    # change target 3 attached order
                    if bridge.get_order_status(trade_key, order_id = target3_target).order_status_code == OrderStatusCode.OPEN:
                        bridge.modify_order(trade_key, order_id = target3_target, quantity= Third_oco_quantity, price1= Third_oco_target)
                        print("Change target 3 attached order 3")

                elif Management_condition == 5:
                    # increase stoploss to BE

                    if (bridge.get_order_status(trade_key, order_id = target1_stop).order_status_code == OrderStatusCode.OPEN and
                        bridge.get_order_status(trade_key,target1_stop).price1 < stopprice):
                        bridge.modify_order(trade_key, order_id = target1_stop, quantity= First_oco_quantity, price1= stopprice)
                        print("Increasing stoploss")

                    if (bridge.get_order_status(trade_key, order_id = target2_stop).order_status_code == OrderStatusCode.OPEN and
                       bridge.get_order_status(trade_key,target2_stop).price1 < stopprice):
                        bridge.modify_order(trade_key, order_id =target2_stop, quantity= Second_oco_quantity, price1= stopprice)
                        print("Increasing stoploss")

                    if (bridge.get_order_status(trade_key, order_id = target3_stop).order_status_code == OrderStatusCode.OPEN and
                       bridge.get_order_status(trade_key,target3_stop).price1 < stopprice):
                        bridge.modify_order(trade_key, order_id = target3_stop, quantity= Third_oco_quantity, price1= stopprice)
                        print("Increasing stoploss")

                elif Management_condition == 6:
                    # Move stoploss because of takeprofit

                    if (bridge.get_order_status(trade_key,order_id=target1_stop).order_status_code == OrderStatusCode.OPEN and round_to_quarters(
                            bridge.get_order_status(trade_key,target1_stop).price1 < BE_price)):
                        bridge.modify_order(trade_key, order_id=target1_stop, quantity=First_oco_quantity, price1=BE_price)
                        print("Move stoploss because of takeprofit order")

                    if (bridge.get_order_status(trade_key, order_id=target2_stop).order_status_code == OrderStatusCode.OPEN and round_to_quarters(
                        bridge.get_order_status(trade_key,target2_stop).price1 < BE_price)):
                        bridge.modify_order(trade_key, order_id=target2_stop, quantity=Second_oco_quantity, price1=BE_price)
                        print("Move stoploss because of takeprofit order")

                    if (bridge.get_order_status(trade_key, order_id=target3_stop).order_status_code == OrderStatusCode.OPEN and round_to_quarters(bridge.get_order_status(trade_key,
                    target3_stop).price1 < BE_price)):
                        bridge.modify_order(trade_key, order_id=target3_stop, quantity=Third_oco_quantity, price1=BE_price)
                        print("Move stoploss because of takeprofit order")

            elif Position_quantity < 0:
                BE_price = round_to_quarters(bridge.get_order_status(trade_key, target1_parent).average_fill_price)

                if Management_condition == 1:
                    bridge.flatten_and_cancel(trade_key)
                    print("Flatten & canceled position ")

                elif Management_condition == 2:
                    # change target attached order
                    if bridge.get_order_status(trade_key,order_id=target1_target).order_status_code == OrderStatusCode.OPEN:
                        bridge.modify_order(trade_key, order_id=target1_target, quantity=First_oco_quantity,price1=First_oco_target)
                        print("Change target attached order 1 ")

                elif Management_condition == 3:
                    # change target 2 attached order

                    if bridge.get_order_status(trade_key,order_id=target2_target).order_status_code == OrderStatusCode.OPEN:
                        bridge.modify_order(trade_key, order_id=target2_target, quantity=Second_oco_quantity,price1=Second_oco_target)
                        print("Change target 2 attached order 2")

                elif Management_condition == 4:
                    # change target 3 attached order
                    if bridge.get_order_status(trade_key,order_id=target3_target).order_status_code == OrderStatusCode.OPEN:
                        bridge.modify_order(trade_key, order_id=target3_target, quantity=Third_oco_quantity,price1=Third_oco_target)
                        print("Change target 3 attached order 3")

                elif Management_condition == 5:
                    # increase stoploss to BE

                    if bridge.get_order_status(trade_key,order_id=target1_stop).order_status_code == OrderStatusCode.OPEN and bridge.get_order_status(
                            trade_key, target1_stop).price1 > stopprice:
                        bridge.modify_order(trade_key, order_id=target1_stop, quantity=First_oco_quantity,price1=stopprice)
                        print("Increasing stoploss")

                    if bridge.get_order_status(trade_key,order_id=target2_stop).order_status_code == OrderStatusCode.OPEN and bridge.get_order_status(
                            trade_key, target2_stop).price1 > stopprice:
                        bridge.modify_order(trade_key, order_id=target2_stop, quantity=Second_oco_quantity,price1=stopprice)
                        print("Increasing stoploss")

                    if bridge.get_order_status(trade_key,order_id=target3_stop).order_status_code == OrderStatusCode.OPEN and bridge.get_order_status(
                            trade_key, target3_stop).price1 > stopprice:
                        bridge.modify_order(trade_key, order_id=target3_stop, quantity=Third_oco_quantity,price1=stopprice)
                        print("Increasing stoploss")

                elif Management_condition == 6:
                    # Move stoploss because of takeprofit

                    if (bridge.get_order_status(trade_key,order_id=target1_stop).order_status_code == OrderStatusCode.OPEN and round_to_quarters(
                            bridge.get_order_status(trade_key, target1_stop).price1 > BE_price)):
                        bridge.modify_order(trade_key, order_id=target1_stop, quantity=First_oco_quantity,price1=BE_price)
                        print("Move stoploss because of takeprofit order")

                    if (bridge.get_order_status(trade_key,order_id=target2_stop).order_status_code == OrderStatusCode.OPEN and round_to_quarters(
                            bridge.get_order_status(trade_key, target2_stop).price1 > BE_price)):
                        bridge.modify_order(trade_key, order_id=target2_stop, quantity=Second_oco_quantity,price1=BE_price)
                        print("Move stoploss because of takeprofit order")

                    if (bridge.get_order_status(trade_key,order_id=target3_stop).order_status_code == OrderStatusCode.OPEN and round_to_quarters(
                            bridge.get_order_status(trade_key,target3_stop).price1 > BE_price)):
                        bridge.modify_order(trade_key, order_id=target3_stop, quantity=Third_oco_quantity,price1=BE_price)
                        print("Move stoploss because of takeprofit order")


    except Exception as e:
        print("Error calculating risk:", e)

Extra risk management rules to stop trading if needed

In [ ]:
def riskmanagement_stop():
    try:

        Position_quantity = global_df[' Position Quantity'].iloc[-1]
        current_time = datetime.now().time()
        trade_time_close = datetime.strptime(Trade_time_close, "%H:%M").time()
        trade_time_open = datetime.strptime(Trade_time_open, "%H:%M").time()
        flatten_positions_time = datetime.strptime(Flatten_positions_time, "%H:%M").time()

        current_pnl = global_df[' Daily Profit/Loss'].iloc[-1] * (tick_value  * 4)

        if current_time <= trade_time_open:
            print(f'{current_time}: Outside of trading hours, open time: {trade_time_open}')
            return True

        if current_time >= trade_time_close:
            print(f'{current_time}: Outside of trading hours, end time: {trade_time_close}')
            return True

        if current_time >= flatten_positions_time and Position_quantity != 0:
            bridge.flatten_and_cancel(trade_key)
            print(f'{current_time}: Flatten Position time: {flatten_positions_time}')
            return True

        if current_pnl >= Profit_target:
            print(f'{current_time}: Hit profit target, stopped trading. Current PnL {current_pnl}')
            return True

        if current_pnl <= (-1) * Maximum_loss_per_day:
                print(f'{current_time}: Hit Max Loss for day, stopped trading. Current PnL {current_pnl}')
                return True


        return False

    except Exception as e:
        print("Error in stopping trading:", e)


Send discord messages, every minute about the current state and when position is taken. Also post a chart every hour to show long / short alerts.



In [ ]:
def Discord_message():
    global last_message_time
    try:
        current_time_discord_message = datetime.now()
        if (global_df['Bias'].iloc[-1] == ':green_circle: Long' and global_df['Bias'].iloc[
            -13] == ':white_circle: Hold') and (datetime.now() - last_message_time).total_seconds() >= Deltabot_interval:
            discord_message = {
                "content": f' Bias flipped from {global_df["Bias"].iloc[-13]} to {global_df["Bias"].iloc[-1]}'
            }
            message = requests.post(webhook_url, data=json.dumps(discord_message),
                                    headers={'Content-Type': 'application/json'})
            last_message_time = current_time_discord_message


        # Check for a bias flip from Hold to Short
        if (global_df['Bias'].iloc[-1] == ':red_circle: Short' and global_df['Bias'].iloc[-13] == ':white_circle: Hold') and (datetime.now() - last_message_time).total_seconds() >= Deltabot_interval:
            discord_message = {
                "content": f' @everyone Bias flipped from {global_df["Bias"].iloc[-13]} to {global_df["Bias"].iloc[-1]}'
            }
            message = requests.post(webhook_url, data=json.dumps(discord_message),
                                    headers={'Content-Type': 'application/json'})
            last_message_time = current_time_discord_message

        if global_df[' Position Quantity'].iloc[-1] != global_df[' Position Quantity'].iloc[-2] and (datetime.now() - last_message_time).total_seconds() >= Deltabot_interval:
            discord_message = {
                "content": f'@ everyone Orderfilled, Quantity changed from {global_df[' Position Quantity'].iloc[-2]}, '
                           f'to {global_df[' Position Quantity'].iloc[-1]} at price: {global_df[' Last'].iloc[-2]}. '
            }
            message = requests.post(webhook_url, data=json.dumps(discord_message),
                                    headers={'Content-type': 'application/json'})
            last_message_time = current_time_discord_message


    except Exception as e:
        print("Error sending discord message:", e)

def Discord_message_minute():
    try:
            bias_value = global_df['Bias'].iloc[-1]
            if bias_value == ':green_circle: Long':
                embed_color = 0x00FF00  # Green
            elif bias_value == ':red_circle: Short':
                embed_color = 0xFF0000  # Red
            elif bias_value == ':white_circle: Hold':
                embed_color = 0xFFFFFF  # White
            else:
                embed_color = 0x000000

            message = {
                "embeds": [{
                    "title": f"Delta Algo, Time: {global_df['DateTime'].iloc[-1]}",
                    "color": embed_color,
                    "fields": [
                        {"name": "Bias", "value": f'{global_df['Bias'].iloc[-1]}', "inline": True},
                        {"name": "Liquidity Direction", "value": f'{global_df['Liquidity Side'].iloc[-1]}', "inline": True},
                        {"name": "Book Average", "value": f'{global_df[' Avg'].iloc[-1]}', "inline": True},
                        {"name": "Delta Direction", "value": f'{option_delta['Position'].iloc[-2]}', "inline": True},
                        {"name": "Delta ROC", "value": f'{option_delta['ROC_15min'].iloc[-2].round(4)}', "inline": True},
                        {"name": "ROC bias", "value": f'{option_delta['Delta'].iloc[-2].round(4)}', "inline": True},
                        {"name": "T1", "value": f'{First_oco_target}', "inline": True},
                        {"name": "T2", "value": f'{Second_oco_target}', "inline": True},
                        {"name": "Stoploss", "value": f'{Stoploss_price}', "inline": True},
                        {"name": "Current Position", "value": f'{global_df[' Position Quantity'].iloc[-1]}', "inline": True},
                        {"name": "RSI Value", "value": f'{global_df[' RSI'].iloc[-1]}', "inline": True},
                        {"name": "Second since extension", "value": f'{global_df['Normal Action Count'].iloc[-1] * 5}', "inline": True}


                    ]
                }]
            }
            discord_message = requests.post(webhook_url, data=json.dumps(message),
                                            headers={'Content-Type': 'application/json'})

    except Exception as e:
        print("Error sending minutely discord message:", e)
        
def determine_bias(row):
    if row[' Liquidity Trend'] == 1 and row['Position'] == 'Long':
        return ':green_circle: Long'
    elif row[' Liquidity Trend'] ==-1 and row['Position'] == 'Short':
        return ':red_circle: Short'
    else:
        return ':white_circle: Hold'

def plot_hourly_chart():
    global hourlychart_df
    try:

        hourlychart_df  = merged_df[[' High', ' Low', ' Open', ' Last', 'Bias', 'Position', 'ROC_15min', 'Delta', ' Liquidity Trend', 'Extension', ' Avg', 'Normal Action Count']].tail(720)
        hourlychart_df.columns = hourlychart_df.columns.str.strip()
        hourlychart_df['Close'] = hourlychart_df['Last']

        # Plot the candlestick chart without specifying ax
        fig, axes = mpf.plot(hourlychart_df, type='line', title='Hourly Chart',
                             style='charles', volume=False, returnfig=True, warn_too_much_data=11800)

        ax = axes[0]

        for i in range(len(hourlychart_df) - 1):
            condition = hourlychart_df['Bias'].iloc[i]
            condition_rsi = hourlychart_df['Extension'].iloc[i]
            condition_rsi_minute_ago = hourlychart_df['Normal Action Count'].iloc[i]
            average_of_book = hourlychart_df['Avg'].iloc[i]

            # Set line color and thickness based on the condition
            if condition == ':green_circle: Long':
                shade_color = 'green'
            elif condition == ':red_circle: Short':
                shade_color = 'red'
            else:
                shade_color = 'white'

            if average_of_book <= round(90):
                line_color = 'red'
                line_width = 1.5

            elif condition_rsi == 'Extended':
                line_color = 'yellow'
                line_width = 1.5

            elif condition_rsi_minute_ago <= 12:
                line_color = 'yellow'
                line_width = 1.5
            else:
                line_color = 'blue'  # Default color if no condition is met
                line_width = 0.5

            # Plot line segment with specific color and thickness
            ax.plot([i, i+1], [hourlychart_df['Close'].iloc[i], hourlychart_df['Close'].iloc[i + 1]],
                    color=line_color, linewidth=line_width)

            # Apply background shading based on the condition
            ax.axvspan(i, i+1, color=shade_color, alpha=0.2)


        buf = io.BytesIO()
        buf_2 = io.BytesIO()
        plt.savefig(buf, format='png')
        plt.savefig(buf_2, format='png')
        buf.seek(0)  #
        buf_2.seek(0)

        # Send the image to the Discord webhook
        response = requests.post(
            webhook_url,
            files={'file': ('hourly_chart.png', buf, 'image/png')}
        )
        response_hourly = requests.post(
            webhook_url_hourly,
            files={'file': ('hourly_chart.png', buf_2, 'image/png')}
        )

    except Exception as e:
        print("Error sending hourly discord chart:", e)

New Calculation for current accepted liquidity


In [ ]:
def Non_periodrange():
    global global_df
    try:
        global_df['Highline'] = 0.0
        global_df['Lowline'] = 0.0
        global_df['Creation Period'] = 0
        tick_spread = 24
        tick_size = 0.25
        tick_range = tick_spread * tick_size

        for index in range(len(global_df)):
            if index == 0:
                global_df.at[index, 'Highline'] = global_df.at[index, ' High'] + tick_range
                global_df.at[index, 'Lowline'] = global_df.at[index, ' Low'] - tick_range
            else:
                # Adjust lines based on the previous index
                if global_df.at[index - 1, 'Lowline'] >= global_df.at[index - 1, ' Low']:
                    global_df.at[index, 'Lowline'] = global_df.at[index - 1, 'Lowline'] - tick_range
                    global_df.at[index, 'Highline'] = global_df.at[index - 1, 'Highline'] - tick_range
                    global_df.at[index, 'Creation Period'] = global_df.at[index - 1, 'Creation Period'] + 1
                elif global_df.at[index - 1, 'Highline'] <= global_df.at[index - 1, ' High']:
                    global_df.at[index, 'Lowline'] = global_df.at[index - 1, 'Lowline'] + tick_range
                    global_df.at[index, 'Highline'] = global_df.at[index - 1, 'Highline'] + tick_range
                    global_df.at[index, 'Creation Period'] = global_df.at[index - 1, 'Creation Period'] + 1
                else:
                    global_df.at[index, 'Lowline'] = global_df.at[index - 1, 'Lowline']
                    global_df.at[index, 'Creation Period'] = global_df.at[index - 1, 'Creation Period']
                    global_df.at[index, 'Highline'] = global_df.at[index - 1, 'Highline']



    except Exception as e:
        print("Error calculating the Nonperiod Range:", e)
        
def Zone_creation_period():
    global zone_df, filled_zone_df, unfilled_zone_df
    try:
        zone_df['Creation Period'] = 0
        zone_df['Zone Direction'] = None
        zone_df[' Open_time'] = pd.to_datetime(zone_df[' Open_time'])

        for i in range(len(zone_df)):
            creation_time = zone_df.at[i, ' Open_time']

            # Find the row in global_df that matches the creation time
            matching_row = global_df[global_df['DateTime'] == creation_time]

            if not matching_row.empty:
                # Get 'Current Period' from the matching row and assign it to 'Creation Period' in unfilled_zone_df
                zone_df.at[i, 'Creation Period'] = matching_row.iloc[0]['Creation Period']
                zone_direction = None
                if zone_df.at[i, ' Value'] >= matching_row.iloc[0][' Last']:
                    zone_df.at[i, 'Zone Direction'] = 'Long'

                elif zone_df.at[i, ' Value'] <= matching_row.iloc[0][' Last']:
                    zone_df.at[i, 'Zone Direction'] = 'Short'

            else:
                # Handle cases where no matching time is found
                zone_df.at[i, 'Creation Period'] = 0
                zone_df.at[i, 'Zone Direction'] = None

        # Initialize empty DataFrames
        filled_zone_df = pd.DataFrame(columns=zone_df.columns)
        unfilled_zone_df = pd.DataFrame(columns=zone_df.columns)
        unfilled_zone_df = zone_df[zone_df[' Close_time'] == ' Not Filled']
        filled_zone_df = zone_df[zone_df[' Close_time'] != ' Not Filled']
        filled_zone_df.loc[:, ' Close_time'] = pd.to_datetime(filled_zone_df[' Close_time'])

    except Exception as e:
        print("Error defining Liquidity period:", e)
        
def Liquidity_trendV2():
    global global_df, zone_df, filled_zone_df, unfilled_zone_df
    try:
        global_df['Last Filled Period'] = 0
        global_df['Liquidity Trend V2'] = 'Flat'
        global_df['DateTime'] = pd.to_datetime(global_df['DateTime'])
        for i in range(len(global_df)):
            Current_period = global_df['Creation Period'].iloc[i]
            Bar_time = global_df['DateTime'].iloc[i]

            if Current_period != 0:
                filtered_df = filled_zone_df[filled_zone_df[' Close_time'] <= Bar_time]

                if not filtered_df.empty:
                    max_time_row = filtered_df.loc[filtered_df[' Close_time'].idxmax()]
                    creation_period = max_time_row['Creation Period']

                    if creation_period <= Current_period - 1:
                        Zone_direction = max_time_row['Zone Direction']
                        if Zone_direction == 'Long':
                            global_df.loc[i, 'Liquidity Trend V2'] = 'Long'

                        elif Zone_direction == 'Short':
                            global_df.loc[i, 'Liquidity Trend V2'] = 'Short'
                    else:
                        global_df.loc[i, 'Liquidity Trend V2'] = global_df.loc[i - 1, 'Liquidity Trend V2']

    except Exception as e:
        print("Error defining Liquidity trend:", e)

Actual loop for the system to run

In [ ]:
def start_update_loop():
    last_minute_run = datetime.now() - timedelta(minutes=1)
    last_hour_run = datetime.now() - timedelta(hours=1)
    deltabot_download = datetime.strptime(Deltabot_downloadtime, "%H:%M").time()
    while True:
        current_time = datetime.now()
        c_time = datetime.now().time()
        if (datetime.now() - last_minute_run).total_seconds() >= Deltabot_interval and c_time >= deltabot_download:
            download_optionbot()
        else:
            print(f'Current Time: {c_time}, is before {deltabot_download}')
        read_csv()
        Option_delta_calculation()
        calculate_zones()
        calculate_liquidity_side()
        Non_periodrange()
        Zone_creation_period()
        Liquidity_trendV2()
        if riskmanagement_stop() == False:
            calculate_risk_management()

        if (datetime.now() - last_hour_run).total_seconds() >= 60 * 60:
            merge_dataframes()
            merged_df['Bias'] = merged_df.apply(determine_bias, axis=1)
            plot_hourly_chart()
            last_hour_run = current_time
        Discord_message()
        if (datetime.now() - last_minute_run).total_seconds() >= Deltabot_interval:
            Discord_message_minute()
            last_minute_run = current_time
        print(f'Liquidity is: {global_df['Liquidity Side'].iloc[-1]}, Deltabot is: {option_delta['Position'].iloc[ -2]}')
        time.sleep(sleep_interval)

# Start the update loop
start_update_loop()